# Query Text Data workflow with "Lazy GraphRAG" retrieval mechanism

Vector-based semantic search is a "best first" search approach for relevant chunks - it simply takes the top k most similar chunks based on embedding distances. It has no sense of the breadth of semantic content in the dataset.
Full [GraphRAG](https://aka.ms/graphrag) is a "breadth first" search approach for relevant content in community summaries - it always queries each community. It has no sense of which communities are best.
ming rapid relevance tests on these summaries in rank order to confirm that only the best community summaries from across the breadth of the dataset are used for answer generation.

Lazy GraphRAG requires no summarization-based index (which is expensive, especially if the user only has a few queries). Instead, it relies purely on an embedding-based index for input text chunks (cheap, fast) augmented by the concept graph of noun-phrase cooccurrences extracted from these chunks (free, fast). It works as follows:

- Given a query, text chunks are ranked using vector-based semantic search.
- Chunk rankings are used to determine community rankings in the concept graph.
- In community rank order, the top k chunks are subject to a rapid relevant test (cheap, fast). If relevant chunks are found, the community is retained, otherwise it is discarded.
- After n communities fail to return any relevant chunks, the "iterative narrowing" process repeats with remaining communities in rank order.
- The process terminates when no more testable chunks exist or a predefined testing budget is reached (e.g., 100 relevance tests).

The advantages of this approach include:

- A balance of best-first (semantic search) and breadth-first (community structure) approaches for any testing budget
- Very fast and almost free - suitable for one-off and exploratory queries before committing to a full indexing
- Easily accommodates dynamic contexts where new texts/chunks are streaming in to the dataset (supported in the current implementation)
- Provides a consistent level of "effort" for a fixed test budget, providing various standards of answers to questions (e.g., global questions) for which there is no ground truth (e.g., 1000 chunks = gold standard, 500 chunks = silver standard, 100 chunks = bronze standard, etc.)


In [1]:
import sys

sys.path.append("..")

In [2]:
import os
import asyncio
import io
import pandas as pd
from toolkit.AI.openai_configuration import OpenAIConfiguration
from toolkit.AI.openai_embedder import OpenAIEmbedder
from toolkit.helpers.constants import CACHE_PATH

import toolkit.query_text_data.input_processor as input_processor
import toolkit.query_text_data.relevance_assessor as relevance_assessor
import toolkit.query_text_data.answer_builder as answer_builder
import toolkit.query_text_data.graph_builder as graph_builder
import toolkit.query_text_data.pattern_detector as pattern_detector
import toolkit.graph.graph_fusion_encoder_embedding as gfee
import toolkit.query_text_data.helper_functions as helper_functions
import nest_asyncio # Necessary to run async code in ipynb

nest_asyncio.apply()

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
# Comment/uncomment the following lines to toggle access to library code updates
import importlib
importlib.reload(input_processor)
importlib.reload(pattern_detector)
importlib.reload(helper_functions)
importlib.reload(relevance_assessor)
importlib.reload(answer_builder)
importlib.reload(graph_builder)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


<module 'toolkit.query_text_data.graph_builder' from 'c:\\Users\\daedge\\Code\\ITK\\intelligence-toolkit\\example_notebooks\\..\\toolkit\\query_text_data\\graph_builder.py'>

In [4]:
# Set up the AI model and embedding model
ai_configuration = OpenAIConfiguration(
    {
        "api_type": "OpenAI",
        "api_key": os.environ["OPENAI_API_KEY"],
        "model": "gpt-4o-2024-08-06",
    }
)

text_embedder = OpenAIEmbedder(
    configuration=ai_configuration,
)

In [5]:
# Provide text inputs as rows of a dataframe
input_path = "../example_outputs/query_text_data/news_articles/news_articles_texts.csv"
file_name = input_path.split("/")[-1]
file_to_chunks = input_processor.process_file_bytes({file_name: open(input_path, 'rb').read()}, input_processor.PeriodOption.NONE)
print("Chunked input text")

Chunked input text


In [6]:
max_cluster_size = 25 # The maximum number of concepts in a leaf-level community in the concept graph
# Process the chunks into the index data structures
(
    cid_to_text,
    text_to_cid,
    period_concept_graphs,
    community_to_concepts,
    concept_to_community,
    concept_to_cids,
    cid_to_concepts,
    previous_cid,
    next_cid,
    period_to_cids,
    node_period_counts,
    edge_period_counts,
    hierarchical_communities
) = input_processor.process_chunks(file_to_chunks=file_to_chunks, max_cluster_size=max_cluster_size)
print(f"Processed chunks")

Processed chunks


In [7]:
# Embed the text chunks
cid_to_vector = await helper_functions.embed_texts(
    cid_to_text=cid_to_text,
    text_embedder=text_embedder,
)
print(f"Embedded chunk text")

Got 501 existing texts
Got 0 new texts
Embedded chunk text


In [8]:
question = "What events are discussed?"
relevance_test_budget = 100 # How many relevance tests are permitted per query. Higher values may provide higher quality results at higher cost
adjacent_search_steps = 1 # How many chunks before and after each relevant chunk to test, once the relevance test budget is near or the search process has terminted
community_relevance_tests = 5 # How many relevance tests to run on each community in turn
relevance_test_batch_size = 5 # How many relevance tests to run in parallel at a time
irrelevant_community_restart = 3 # When to restart testing communities in relevance order

# Mine relevant chunks to the question
async def mine_relevant_chunks():
    (
        relevant_cids,
        chunk_progress,
    ) = await relevance_assessor.detect_relevant_chunks(
        ai_configuration=ai_configuration,
        question=question,
        cid_to_text=cid_to_text,
        cid_to_concepts=cid_to_concepts,
        cid_to_vector=cid_to_vector,
        community_to_concepts=community_to_concepts,
        concept_to_community=concept_to_community,
        previous_cid=previous_cid,
        next_cid=next_cid,
        embedder=text_embedder,
        embedding_cache=CACHE_PATH,
        select_logit_bias=5,
        adjacent_search_steps=adjacent_search_steps,
        relevance_test_budget=relevance_test_budget,
        community_relevance_tests=community_relevance_tests,
        relevance_test_batch_size=relevance_test_batch_size,
        irrelevant_community_restart=irrelevant_community_restart,
    )
    return relevant_cids, chunk_progress

if __name__ == "__main__":
    relevant_cids, chunk_progress = await mine_relevant_chunks()
    print(f"Mined relevant chunks")
    


New loop after 0 tests
Assessing relevance for topic 108 with 5 chunks


100%|██████████| 5/5 [00:01<00:00,  4.26it/s]


Batch 1 of 1: 5 relevant chunks
Community 108: True
Assessing relevance for topic 106 with 5 chunks


100%|██████████| 5/5 [00:01<00:00,  3.52it/s]


Batch 1 of 1: 5 relevant chunks
Community 106: True
Assessing relevance for topic 97 with 5 chunks


...

Batch 1 of 1: 0 relevant chunks
Community 98: False
Assessing relevance for topic 63 with 5 chunks


0it [00:00, ?it/s]


Batch 1 of 1: 0 relevant chunks
Community 63: False
Assessing relevance for topic 157 with 2 chunks


0it [00:00, ?it/s]

Batch 1 of 1: 0 relevant chunks
Community 157: False
0 successive irrelevant communities; breaking
Assessing relevance for neighbours with 0 chunks
[('topic 158', 153, 'Yes'), ('topic 38', 341, 'Yes'), ('topic 38', 468, 'Yes')]
**Chunks relevant/tested: 96/100** (topic 108: 5/5; topic 106: 5/5; topic 97: 5/5; topic 54: 5/5; topic 56: 4/4; topic 105: 4/5; topic 103: 5/5; topic 102: 5/5; topic 53: 5/5; topic 156: 5/5; topic 57: 5/5; topic 51: 5/5; topic 122: 5/5; topic 104: 3/3; topic 107: 5/5; topic 96: 4/5; topic 4: 4/5; topic 46: 4/5; topic 110: 5/5; topic 150: 5/5; topic 158: 1/1; topic 38: 2/2)
Mined relevant chunks


In [9]:
answer_batch_size = 10 # How many chunks to integrate into the answer at a time
# Generate an extended answer to the question, which could then be summarized
(
    partial_answers,
    answer_progress,
) = answer_builder.answer_question(
    ai_configuration=ai_configuration,
    question=question,
    relevant_cids=relevant_cids,
    cid_to_text=cid_to_text,
    answer_batch_size=answer_batch_size,
)
print(f"Answered question")

Answered question


In [10]:
# Output some process summarry information
print(chunk_progress)
print(answer_progress)

**Chunks relevant/tested: 96/100** (topic 108: 5/5; topic 106: 5/5; topic 97: 5/5; topic 54: 5/5; topic 56: 4/4; topic 105: 4/5; topic 103: 5/5; topic 102: 5/5; topic 53: 5/5; topic 156: 5/5; topic 57: 5/5; topic 51: 5/5; topic 122: 5/5; topic 104: 3/3; topic 107: 5/5; topic 96: 4/5; topic 4: 4/5; topic 46: 4/5; topic 110: 5/5; topic 150: 5/5; topic 158: 1/1; topic 38: 2/2)
**Chunks referenced/relevant: 86/96** (answer 1: 8/10, answer 2: 8/10, answer 3: 10/10, answer 4: 10/10, answer 5: 10/10, answer 6: 10/10, answer 7: 9/10, answer 8: 8/10, answer 9: 8/10, answer 10: 5/6)


In [11]:
# Output the final extended answer
final_answer = partial_answers[-1]
print(final_answer)

# Key Global Events and Discussions in 2023-2024

*In response to: What events are discussed?*

## Introduction

This report provides an overview of significant global events and discussions that took place in 2023 and 2024, highlighting humanitarian efforts, economic forums, cultural festivals, climate action, and security challenges. The events are grouped thematically as follows:

- **Humanitarian and Cultural Events**
  - Humanitarian Aid Efforts in Indonesia
  - Miami Spring Festival: A Cultural Extravaganza
  - Annual Community Fair Brings Neighbors Together
  - DJ Alex Headlines Berlin Music Festival with Electrifying Performance
  - Annual Music Festival Thrills Crowds at Sunset Park
  - Annual Charity Run Set to Take Place at Riverfront Park
  - Hope Fundraiser Gala Raises Millions for Charity
  - Beyoncé to Perform at Sydney Opera House
  - Zoe Lin Rocks the Stage at SXSW
  - Riverside Park Cleanup Drive: A Community Unites for the Environment
  - Annual Food Festival Returns